# Plotting Geo Data

In [1]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, CustomJS, HoverTool, CategoricalColorMapper
from bokeh.palettes import Accent
from bokeh.models.widgets import Toggle, Button, CheckboxGroup, DataTable, DateFormatter, TableColumn
from bokeh.layouts import row as bokeh_row
from bokeh.layouts import column
from bokeh.models.glyphs import Patches

import math
import json
import pandas as pd
import os
import re
import numpy as np
from ast import literal_eval

# data sources

#### https://data.stadt-zuerich.ch/

* Future population of zurich (download link) https://www.stadt-zuerich.ch/content/dam/stzh/prd/Deutsch/Statistik/Themen/Bevoelkerung/BEV340T3401_Zukuenftige_Bevoelkerungsentwicklung-Veraenderung_nach-Stadtkreis-Stadtquartier.xlsx

* Zurich Kreis areas https://de.wikipedia.org/wiki/Stadtteile_der_Stadt_Z%C3%BCrich


* Locations of automatic counting devices for pedestrian and bicycle traffic
    https://data.stadt-zuerich.ch/dataset/verkehrszaehlungen-standorte-velo-fussgaenger
    
* Automatic counting data «Pedestrian traffic» - quarter-hourly values
    https://data.stadt-zuerich.ch/dataset/verkehrszaehlungen-werte-fussgaenger-velo


* Fusswegnetz
    https://data.stadt-zuerich.ch/dataset/fussweg
    
* Official color of Stadt Zuerich
    https://www.stadt-zuerich.ch/prd/de/index/stadtarchiv/RechercheBenutzung/recherche/beliebteforschungsthemen/das_wappen_der_stadtzuerich.html
 
* color palette blender/generator: https://meyerweb.com/eric/tools/color-blend/#0066CC:D6E2FF:4:hex

### Functions


In [5]:
def round_up(num, sig_figs):
    
    return math.ceil(num / (10**sig_figs)) * (10**sig_figs)

In [6]:
def round_down(num, sig_figs):
    
    return math.floor(num / (10**sig_figs)) * (10**sig_figs)

In [7]:
def extract_ints(in_string):
    out_string = re.sub('[^0-9]','', in_string)
    if len(out_string):
        return int(out_string)
    else:
        return np.nan

In [8]:
def merc_proj(coords):
    """
    coords: a tuple containing coordinates ordered as (lat, lon)
    """
    lat, lon = coords    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(max(0, math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0))) * scale
    
    return x, y

In [9]:
def deg2num(lat_deg, lon_deg, zoom):
      lat_rad = math.radians(lat_deg)
      n = 2.0 ** zoom
      xtile = int((lon_deg + 180.0) / 360.0 * n)
      ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
      return (xtile, ytile)

## Load Data

In [10]:
folder = ''

### Zurich Kreis Buro

In [11]:
zur_long= 8.5417
zur_lat = 47.3769
coord = (zur_lat, zur_long)
zur_x, zur_y = merc_proj(coord)

print(zur_x,', ', zur_y)

950857.6945089049 ,  6003812.204877849


In [12]:
fname = 'stadtkreis.json'
fpath = os.path.join(folder,fname )

with open(fpath, 'r') as f:
    kreis_json = json.load(f) 

In [13]:
kreis_json.keys()

dict_keys(['name', 'type', 'features'])

In [14]:
kreis_json['features'][0]['properties']['Bezeichnnung']

'Kreis 9'

In [15]:
# check that all coordinates are polygons
for f in kreis_json['features']:
    print(f['geometry']['type'])

Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon


In [16]:
df_kreis_coord = pd.DataFrame()

for feature in kreis_json['features']:
    kreis_dict = {}
    kreis_dict['kreis'] = re.sub('[^0-9]','', feature['properties']['Bezeichnnung'])
    kreis_dict['coords'] = feature['geometry']['coordinates'][0]
    kreis_dict['coords'] = [(el[1], el[0]) for el in  kreis_dict['coords']]
    df_kreis_coord = df_kreis_coord.append(kreis_dict, ignore_index=True)
    
df_kreis_coord = df_kreis_coord.sort_values('kreis').set_index('kreis')
df_kreis_coord

,coords
kreis,
1,"[(47.3768651828916, 8.54875812345879), (47.376..."
10,"[(47.4042325281943, 8.47819887163832), (47.404..."
11,"[(47.4221066631867, 8.48799434845604), (47.422..."
12,"[(47.4156460170665, 8.56324622139862), (47.415..."
2,"[(47.3354317915205, 8.54297384152714), (47.335..."
3,"[(47.3684680042063, 8.52761485172454), (47.368..."
4,"[(47.3909036731392, 8.50236067762573), (47.390..."
5,"[(47.3949271514424, 8.50561161160157), (47.394..."
6,"[(47.4012469984016, 8.55559467141438), (47.401..."


In [17]:
merc_proj(df_kreis_coord.loc['1']['coords'][0])

(951643.4012182932, 6003806.481342967)

In [18]:
def ser_list_of_coords2xy_cols(ser):
    """
    converts a series which contains lists of coordinates(as tuples of lat, long)
    into 2 series of lists of x and y points respectively
    uses the mercator projection.
    
    """
    df = pd.DataFrame()
    
    df['x_ys'] = ser.apply(lambda x: [merc_proj(el) for el in x])
    df['xs'] = df['x_ys'].apply(lambda lst: [x for x,y in lst])
    df['ys'] = df['x_ys'].apply(lambda lst: [y for x,y in lst])
    
    
    return df['xs'], df['ys']

In [19]:
df_kreis_coord['xs'], df_kreis_coord['ys'],  = ser_list_of_coords2xy_cols(df_kreis_coord['coords'])
df_kreis_coord.head()

,coords,xs,ys
kreis,,,
1,"[(47.3768651828916, 8.54875812345879), (47.376...","[951643.4012182932, 951643.0115419946, 951642....","[6003806.481342967, 6003804.9435209455, 600380..."
10,"[(47.4042325281943, 8.47819887163832), (47.404...","[943788.7812348843, 943787.6611020037, 943787....","[6008306.526868095, 6008306.586163752, 6008310..."
11,"[(47.4221066631867, 8.48799434845604), (47.422...","[944879.2087263103, 944881.4479297944, 944892....","[6011246.854580052, 6011251.027597851, 6011272..."
12,"[(47.4156460170665, 8.56324622139862), (47.415...","[953256.2089035183, 953343.3812519693, 953377....","[6010183.951369027, 6010213.221760895, 6010224..."
2,"[(47.3354317915205, 8.54297384152714), (47.335...","[950999.4978990573, 950986.0201940133, 950976....","[5996997.98142676, 5996988.238234238, 5996981...."


In [20]:
#df_kreis_coord.to_csv('kreis_coordinates.csv')

### load Population data

In [21]:
fname = 'BEV340T3401_Zukuenftige_Bevoelkerungsentwicklung-Veraenderung_nach-Stadtkreis-Stadtquartier.xlsx'
fpath = os.path.join(folder,fname)

df_pop = pd.read_excel(fpath, sheet_name='2018, 2035', header=8)
df_pop = df_pop.rename(columns={'Unnamed: 0': 'area', 'Veränderung\n(%)':'percentage_change'})
df_pop.columns = [str(col) for col in df_pop.columns]
df_pop['kreis'] = df_pop.area.apply(lambda x: extract_ints(x))
#df_pop['kreis'] = df_pop['kreis'].ffill()
df_pop['pop_diff'] = df_pop['2035'] - df_pop['2018']


In [22]:
df_pop = df_pop[['kreis', 'area', '2018', '2035', 'percentage_change', 'pop_diff']]

In [23]:
df_pop.sort_values(by=['pop_diff','percentage_change'], ascending=False).head()

,kreis,area,2018,2035,percentage_change,pop_diff
0,NaN,Ganze Stadt,428737,504700,17.7,75963
35,11.0,Kreis 11,75344,93800,24.6,18456
39,12.0,Kreis 12,32483,43600,34.2,11117
29,9.0,Kreis 9,55765,65800,18.0,10035
38,NaN,Seebach,25568,34900,36.5,9332


In [24]:
df_pop = df_pop.loc[~df_pop.kreis.isna()]
df_pop['kreis'] = df_pop['kreis'].astype(int).astype(str)
df_pop = df_pop.set_index('kreis')

In [25]:
df_pop

,area,2018,2035,percentage_change,pop_diff
kreis,,,,,
1,Kreis 1,5750,5900,1.8,150
2,Kreis 2,34877,40700,16.7,5823
3,Kreis 3,50569,57400,13.6,6831
4,Kreis 4,28729,32700,13.9,3971
5,Kreis 5,15579,19100,22.7,3521
6,Kreis 6,34321,39000,13.5,4679
7,Kreis 7,38191,43000,12.6,4809
8,Kreis 8,16788,19200,14.3,2412
9,Kreis 9,55765,65800,18.0,10035


## Kreis Areas

In [26]:
df_kreis_area = pd.read_excel('kreis_areas.xlsx')
df_kreis_area['kreis'] =df_kreis_area['kreis'].astype(str) 
df_kreis_area = df_kreis_area.set_index('kreis')
df_kreis_area

,area (km2)
kreis,
1,1.80
2,11.07
3,8.65
4,2.90
5,1.99
6,5.10
7,15.02
8,4.81
9,12.07


## Load House Price Data

In [27]:
fname = 'rent_prices_per_kreis_zurich.csv'
fpath = os.path.join(folder, fname)

df_kreis_rent = pd.read_csv(fpath)
df_kreis_rent['area'] = df_kreis_rent['Kreis']
df_kreis_rent = df_kreis_rent.rename(columns={'Kreis':'kreis'})
df_kreis_rent['kreis'] =df_kreis_rent['kreis'].apply( extract_ints).astype(str)
df_kreis_rent = df_kreis_rent.set_index('kreis')
df_kreis_rent

,pro_quadrat_meter_2013_2017,area
kreis,,
1,29.0,Kreis 1
2,22.8,Kreis 2
3,23.3,Kreis 3
4,23.4,Kreis 4
5,24.2,Kreis 5
6,23.6,Kreis 6
7,23.0,Kreis 7
8,25.0,Kreis 8
9,21.6,Kreis 9


### Merge kreis dfs

In [28]:
df_kreis = df_pop.merge(df_kreis_coord, left_index=True, right_index=True)
df_kreis = df_kreis.merge(df_kreis_area, left_index=True, right_index=True)
df_kreis = df_kreis.merge(df_kreis_rent['pro_quadrat_meter_2013_2017'] , left_index=True, right_index=True)

In [29]:
df_kreis['pop_dens_2018'] = df_kreis['2018'] / df_kreis['area (km2)']
df_kreis['pop_dens_2018'] = df_kreis['pop_dens_2018'].apply(np.floor)
df_kreis['pop_dens_2035'] = df_kreis['2035'] / df_kreis['area (km2)']
df_kreis['pop_dens_2035'] = df_kreis['pop_dens_2035'].apply(np.floor)

In [30]:
df_kreis.head()

,area,2018,2035,percentage_change,pop_diff,coords,xs,ys,area (km2),pro_quadrat_meter_2013_2017,pop_dens_2018,pop_dens_2035
kreis,,,,,,,,,,,,
1,Kreis 1,5750,5900,1.8,150,"[(47.3768651828916, 8.54875812345879), (47.376...","[951643.4012182932, 951643.0115419946, 951642....","[6003806.481342967, 6003804.9435209455, 600380...",1.80,29.0,3194.0,3277.0
2,Kreis 2,34877,40700,16.7,5823,"[(47.3354317915205, 8.54297384152714), (47.335...","[950999.4978990573, 950986.0201940133, 950976....","[5996997.98142676, 5996988.238234238, 5996981....",11.07,22.8,3150.0,3676.0
3,Kreis 3,50569,57400,13.6,6831,"[(47.3684680042063, 8.52761485172454), (47.368...","[949289.7429751329, 949225.292438578, 949160.8...","[6002426.191448782, 6002363.777099468, 6002301...",8.65,23.3,5846.0,6635.0
4,Kreis 4,28729,32700,13.9,3971,"[(47.3909036731392, 8.50236067762573), (47.390...","[946478.4611740486, 946535.3303605559, 946560....","[6006114.555292751, 6006090.378270911, 6006077...",2.90,23.4,9906.0,11275.0
5,Kreis 5,15579,19100,22.7,3521,"[(47.3949271514424, 8.50561161160157), (47.394...","[946840.3534888417, 946863.4846444319, 946886....","[6006776.170360912, 6006772.468790983, 6006769...",1.99,24.2,7828.0,9597.0


## Binning the pop densities

In [83]:
max_val = df_kreis[['pop_dens_2018','pop_dens_2035']].max().max()
min_val = df_kreis[['pop_dens_2018','pop_dens_2035']].min().max()

num_bins = 5
labels = [0,] + list(np.arange(num_bins-1)+1)
print(labels)
sig_figs = int(math.log10((max_val - min_val) / num_bins))
bin_size = round_up((max_val - min_val) / num_bins, sig_figs)

bins =[0] + list(bin_size * (np.arange(num_bins)+2))
bins

[0, 1, 2, 3, 4]


[0, 4000, 6000, 8000, 10000, 12000]

In [84]:
df_kreis['binned_pop_dens_2018'] = pd.cut(df_kreis['pop_dens_2018'], bins=bins, labels=labels )
df_kreis['binned_pop_dens_2035'] = pd.cut(df_kreis['pop_dens_2035'], bins=bins, labels=labels)  

In [85]:
df_kreis['pop_dens_2018']

kreis
1     3194.0
2     3150.0
3     5846.0
4     9906.0
5     7828.0
6     6729.0
7     2542.0
8     3490.0
9     4620.0
10    4437.0
11    5614.0
12    5441.0
Name: pop_dens_2018, dtype: float64

In [88]:
df_kreis[['pop_dens_2018','binned_pop_dens_2018']]

,pop_dens_2018,binned_pop_dens_2018
kreis,,
1,3194.0,0
2,3150.0,0
3,5846.0,1
4,9906.0,3
5,7828.0,2
6,6729.0,2
7,2542.0,0
8,3490.0,0
9,4620.0,1


In [89]:
df_kreis[['pop_dens_2035','binned_pop_dens_2035']]

,pop_dens_2035,binned_pop_dens_2035
kreis,,
1,3277.0,0
2,3676.0,0
3,6635.0,2
4,11275.0,4
5,9597.0,3
6,7647.0,2
7,2862.0,0
8,3991.0,0
9,5451.0,1


#### OUTPUT TO CSV

In [90]:
df_kreis.head()

,area,2018,2035,percentage_change,pop_diff,coords,xs,ys,area (km2),pro_quadrat_meter_2013_2017,pop_dens_2018,pop_dens_2035,binned_pop_dens_2018,binned_pop_dens_2035
kreis,,,,,,,,,,,,,,
1,Kreis 1,5750,5900,1.8,150,"[(47.3768651828916, 8.54875812345879), (47.376...","[951643.4012182932, 951643.0115419946, 951642....","[6003806.481342967, 6003804.9435209455, 600380...",1.80,29.0,3194.0,3277.0,0,0
2,Kreis 2,34877,40700,16.7,5823,"[(47.3354317915205, 8.54297384152714), (47.335...","[950999.4978990573, 950986.0201940133, 950976....","[5996997.98142676, 5996988.238234238, 5996981....",11.07,22.8,3150.0,3676.0,0,0
3,Kreis 3,50569,57400,13.6,6831,"[(47.3684680042063, 8.52761485172454), (47.368...","[949289.7429751329, 949225.292438578, 949160.8...","[6002426.191448782, 6002363.777099468, 6002301...",8.65,23.3,5846.0,6635.0,1,2
4,Kreis 4,28729,32700,13.9,3971,"[(47.3909036731392, 8.50236067762573), (47.390...","[946478.4611740486, 946535.3303605559, 946560....","[6006114.555292751, 6006090.378270911, 6006077...",2.90,23.4,9906.0,11275.0,3,4
5,Kreis 5,15579,19100,22.7,3521,"[(47.3949271514424, 8.50561161160157), (47.394...","[946840.3534888417, 946863.4846444319, 946886....","[6006776.170360912, 6006772.468790983, 6006769...",1.99,24.2,7828.0,9597.0,2,3


## Feature locations

In [91]:
fname = 'data_clean.csv'
fpath = os.path.join(folder, fname)
df_feat = pd.read_csv(fpath)
if 'Unnamed: 0' in df_feat.columns:
    df_feat = df_feat.drop(columns=['Unnamed: 0',])

df_feat['coordinates'] = df_feat.apply(lambda row: (row['lat'], row['lon']), axis=1)
df_feat['circle'] = df_feat['circle'].apply(literal_eval)


In [92]:
 df_feat.head()

,name,lat,lon,criteria,coordinates,coord_eucl,id,kreis,circle
0,Coop Bahnhofbrücke,47.376732,8.542161,competitor,"(47.3767316, 8.542160800000001)","(950908.9905302625, 6003784.521876661)",c9a7f9b6-2518-4fd7-a94b-a70f7e15193f,kreis_1,"[(951308.9905302625, 6003784.521876661), (9513..."
1,Migros Wengihof,47.375020,8.522895,Migros,"(47.37502, 8.522895)","(948764.3314845373, 6003503.159761649)",48df0345-fc0e-464a-9857-5f9d0903f85f,kreis_4,"[(949164.3314845373, 6003503.159761649), (9491..."
2,Aldi,47.340967,8.530601,competitor,"(47.3409666, 8.530600699999999)","(949622.1260847431, 5997907.173911596)",6b999d56-282d-42b9-a91a-0738ec34b816,kreis_2,"[(950022.1260847431, 5997907.173911596), (9500..."
3,Coop Wollishofen,47.340070,8.530546,competitor,"(47.3400698, 8.530546099999999)","(949616.0480405457, 5997759.851807205)",edf18b68-1295-4232-92fd-98bee32f78ba,kreis_2,"[(950016.0480405457, 5997759.851807205), (9500..."
4,Migros Wollishofen,47.344749,8.529981,Migros,"(47.3447487, 8.5299814)","(949553.1859240949, 5998528.507121279)",7a9950ef-1b82-454e-891f-88a1ba70f3b5,kreis_2,"[(949953.1859240949, 5998528.507121279), (9499..."


In [93]:
df_feat['circle_xs'] = df_feat['circle'].apply(lambda lst: [x for x,y in lst])
df_feat['circle_ys'] = df_feat['circle'].apply(lambda lst: [y for x,y in lst])

In [94]:
def coord_col2_xy_cols(ser):
    """takes  a series of coordinates (stored as tuples) and returns 2 columns for x and y points repectively
    uses mercator projcetion"""
    
    df = pd.DataFrame()
    df['x_ys'] = ser.apply(merc_proj)
    df['x'] = df['x_ys'].apply(lambda x: x[0])  # + 725000
    df['y'] = df['x_ys'].apply(lambda x: x[1])
    
    return df['x'], df['y']

In [95]:
df_feat['x'], df_feat['y'] = coord_col2_xy_cols(df_feat['coordinates'])

In [96]:
df_feat.head()

,name,lat,lon,criteria,coordinates,coord_eucl,id,kreis,circle,circle_xs,circle_ys,x,y
0,Coop Bahnhofbrücke,47.376732,8.542161,competitor,"(47.3767316, 8.542160800000001)","(950908.9905302625, 6003784.521876661)",c9a7f9b6-2518-4fd7-a94b-a70f7e15193f,kreis_1,"[(951308.9905302625, 6003784.521876661), (9513...","[951308.9905302625, 951307.0644209314, 951301....","[6003784.521876661, 6003745.31502053, 6003706....",950908.990530,6.003785e+06
1,Migros Wengihof,47.375020,8.522895,Migros,"(47.37502, 8.522895)","(948764.3314845373, 6003503.159761649)",48df0345-fc0e-464a-9857-5f9d0903f85f,kreis_4,"[(949164.3314845373, 6003503.159761649), (9491...","[949164.3314845373, 949162.4053752063, 949156....","[6003503.159761649, 6003463.952905517, 6003425...",948764.331485,6.003503e+06
2,Aldi,47.340967,8.530601,competitor,"(47.3409666, 8.530600699999999)","(949622.1260847431, 5997907.173911596)",6b999d56-282d-42b9-a91a-0738ec34b816,kreis_2,"[(950022.1260847431, 5997907.173911596), (9500...","[950022.1260847431, 950020.199975412, 950014.4...","[5997907.173911596, 5997867.967055464, 5997829...",949622.126085,5.997907e+06
3,Coop Wollishofen,47.340070,8.530546,competitor,"(47.3400698, 8.530546099999999)","(949616.0480405457, 5997759.851807205)",edf18b68-1295-4232-92fd-98bee32f78ba,kreis_2,"[(950016.0480405457, 5997759.851807205), (9500...","[950016.0480405457, 950014.1219312146, 950008....","[5997759.851807205, 5997720.644951073, 5997681...",949616.048041,5.997760e+06
4,Migros Wollishofen,47.344749,8.529981,Migros,"(47.3447487, 8.5299814)","(949553.1859240949, 5998528.507121279)",7a9950ef-1b82-454e-891f-88a1ba70f3b5,kreis_2,"[(949953.1859240949, 5998528.507121279), (9499...","[949953.1859240949, 949951.2598147638, 949945....","[5998528.507121279, 5998489.300265147, 5998450...",949553.185924,5.998529e+06


#### calculate x & y's for the circle coordinates

In [97]:
#df_feat_plot = df_feat[['criteria','name','kreis','x','y', 'circle', ]]

# Plot

In [98]:
ef_zh_coords = [(47.388259, 8.468802),
        (47.418613, 8.480302),
        (47.425864, 8.502571),
        (47.438234, 8.575379),
        (47.350885, 8.568362),
        (47.337249, 8.522954)]

ef_zh_coords_xys = [merc_proj(coord) for coord in ef_zh_coords]
ef_zh_coords_xys

[(942742.7262690568, 6005679.696502305),
 (944022.9004131794, 6010672.06215552),
 (946501.8741536548, 6011865.07029146),
 (954606.8236393315, 6013900.687657055),
 (953825.6947724351, 5999536.689132897),
 (948770.8993344943, 5997296.480306304)]

In [99]:
xs = [el[0] for el in ef_zh_coords_xys]
ys = [el[1] for el in ef_zh_coords_xys]
ys

[6005679.696502305,
 6010672.06215552,
 6011865.07029146,
 6013900.687657055,
 5999536.689132897,
 5997296.480306304]

In [100]:
wint = (47.4988, 8.7237)
wint_x, wint_y = merc_proj(wint)

brem = (47.3492, 8.3398)
brem_x, brem_y = merc_proj(brem)

In [101]:
list(df_feat['criteria'].unique())

['competitor', 'Migros', 'sport', 'station', 'school', 'parking']

In [102]:
pt_size_dict = { 'Migros':10,'competitor':8, 'sport':4, 'station':6, 'school':4, 'parking':4}
df_feat['pt_sizes'] = df_feat['criteria'].map(pt_size_dict)
df_feat[['criteria','pt_sizes']].head()

,criteria,pt_sizes
0,competitor,8
1,Migros,10
2,competitor,8
3,competitor,8
4,Migros,10


# Plotting

In [103]:
df_feat.criteria.unique()

array(['competitor', 'Migros', 'sport', 'station', 'school', 'parking'],
      dtype=object)

### CRITERIA COLORS

In [104]:
colors_pts = ["#FF6600", "#000000", "#3cff00", "#bbff8f","#E7F58A","#ffa8d7"  ]
cats = ["Migros", "competitor", 'station','sport','school','parking' ]

color_map_dict = dict(zip(cats,colors_pts))
df_feat['colors_mapped'] = df_feat['criteria'].map(color_map_dict)
df_feat[['criteria','colors_mapped']].head()

,criteria,colors_mapped
0,competitor,#000000
1,Migros,#FF6600
2,competitor,#000000
3,competitor,#000000
4,Migros,#FF6600


In [107]:

output_file("migros_challenge.html")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

x_size = 1000000
y_size = 1000000

# range bounds supplied in web mercator coordinates
p = figure( x_axis_type="mercator", y_axis_type="mercator", plot_width=750, plot_height=500)
p.add_tile(tile_provider)

#p.circle(x = zur_x, y = zur_y, color='red', size=5, alpha=0.6)
#p.circle(x = wint_x, y = wint_y, size=10, alpha=0)
#p.circle(x = brem_x, y = brem_y, size=10, alpha=0)

# Population Color Formatting
#purple_colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]
blue_colors = ['#D6E2FF', '#A1C3F2', '#6BA4E6', '#3685D9', '#0066CC']
colors = blue_colors

kries_names = [str(int(x)) for x in list(df_kreis.index)]
kries_pop2018 = list(df_kreis['2018'])
kries_pop2018dens = list(df_kreis['pop_dens_2018'])

kries_pop2018_colors = [colors[int(pop * (len(colors)-1) / df_kreis['2018'].max())] for pop in df_kreis['2018']]
kries_pop2018dens_colors = [colors[ind] for ind in df_kreis.binned_pop_dens_2018]  # int(pop * (len(colors)-1) / df_kreis['pop_dens_2018'].max())

kries_pop2035dens = list(df_kreis['pop_dens_2035'])
kries_pop2035_colors = [colors[int(pop * (len(colors)-1) / df_kreis['2035'].max())] for pop in df_kreis['2035']]
kries_pop2035dens_colors = [colors[ind] for ind in df_kreis.binned_pop_dens_2035]
kries_pop2035dens_colors
#PATCHES USING A ColumndDataSource

#source_kreis = ColumnDataSource(df_kreis)
# patches are not visible to hovertools in bokeh
# = p.patch('xs','ys', alpha=.7, line_width=2, source=source_kreis)

#PATCHES USING A LOOP
#add patch glyphs for kreis
for index, row in df_kreis.iterrows():
    kreis2018dens =  p.patch(  row['xs'], row['ys'], alpha=0.4, line_width=2, legend='pop dens 2018', fill_color = kries_pop2018dens_colors[int(index)-1]) # 

for index, row in df_kreis.iterrows():
    kreis2035dens =  p.patch(  row['xs'], row['ys'], alpha=0.4, line_width=2, legend='pop dens 2035', fill_color = kries_pop2035dens_colors[int(index)-1]) # 
###################################################3
#add glyphs for points of interest

source = ColumnDataSource(df_feat)

categories = list(df_feat['criteria'].unique())
#color_mapper = CategoricalColorMapper(factors=categories, palette=Accent[len(categories)])

#category

# MIGROS ORANGE "#FF6600"
color_mapper_pts = CategoricalColorMapper(palette=colors_pts,
                                          factors=cats)
#for index, row in df_feat.iterrows():
    
#circle_patches = Patches(xs="circles_xs", ys="circles_ys", fill_alpha=0.3, fill_color = dict(field='criteria', transform=color_mapper_pts))
#circle_glyphs = p.add_glyph(source, circle_patches)

#### ADD INVISIBLE GLPYHs WITH CDS DATA
sizes = list(df_feat['pt_sizes'])
points = p.circle(x= 'x', y='y', source=source, legend='criteria', alpha=0.9, size='pt_sizes', color = dict(field='criteria', transform=color_mapper_pts) )


#### ADD VISIBLE GLPYHs WITH FOR LOOP TO ENABLE INTERACTIVE legend

for index, row in df_feat.iterrows():
    circle_patch_vis =  p.patch(row["circle_xs"],row["circle_ys"], fill_alpha=0.3, fill_color = row['colors_mapped'], legend=row['criteria'])



p.legend.location = "bottom_right"
p.legend.click_policy = "hide"

#hovertool to display additional cargo info
p_hover = HoverTool(
        tooltips = [ ("feature","@criteria"),
                   ('description', '@name'),
                   ('kreis', '@kreis')], 
        #formatters = {"planned_arrival" : "datetime", "planned_departure": "datetime" },
        mode = 'mouse')

p.add_tools(p_hover)

checkbox_group = CheckboxGroup(
        labels=["Migros", "Competitors", "Stations"], active=[0])

# JavaScript Callback to link toggle button with visible property of the previous schedule
code = '''\

crcl_glyphs.visible = toggle.active

'''

#popdens2018.visible = toggle.active
#popdens2035.hidden = toggle.active

callback1 = CustomJS(code=code, args={})
toggle1 = Toggle(label="Show/Hide points", button_type="primary", callback=callback1)
#callback1.args = {'toggle': toggle1, 'crcl_glyphs': circle_glyphs}



for data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    p.line(df['date'], df['close'], line_width=2, color=color, alpha=0.8,
           muted_color=color, muted_alpha=0.2, legend=name)




In [108]:
datatable_field_list = ['area','2018', '2035', 'pop_diff','percentage_change', 'pop_dens_2018', 'pop_dens_2035','pro_quadrat_meter_2013_2017'  ]
titles = ['Kreis',"pop. 2018","pop. 2035",'pop. change', '%','pop_dens_2018', 'pop_dens_2035','Rent Price (CHF/m2)']
field_title_dict = dict(zip(datatable_field_list,titles))

In [109]:

source_table = ColumnDataSource(df_kreis)
columns = [TableColumn(field=fld_name, title=ttl) for fld_name, ttl in field_title_dict.items()]
data_table = DataTable(source=source_table, columns=columns, width=650, height=500)

In [110]:
output_file("migros_vis.html")

show(bokeh_row(column(p, ),data_table))  #toggle1
